In [1]:
import pandas as pd
import numpy as np
from mapping import Mapping
import time
fig_prefix = "../figuires/2018-07-22-jw-weaselball_analysis"
data_prefix = "../data/2018-07-22-jw-weaselball_analysis_"

In [2]:
start_time = time.time()
FLOAT_ERROR_TOLERANCE = 0.00000000001 #See IEEE 754 for why a floating point is never perfect
df_strings = ['../data/08-28-2018_03-43-02.csv']
frames = []
for csv in df_strings:
    temp = pd.read_csv(csv,index_col=False )
    frames.append(temp)
df = pd.concat(frames,ignore_index=True)
print(df.shape)
df = df.drop(columns=['Time'])
df = df.dropna() #Get rid of any rows with NA in it.
df = df.apply(pd.to_numeric)
df.head(10)

(20433114, 8)


,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
0,0,0.000000,0.000000,0.000000,3,1,0
1,0,0.052917,0.030211,0.023418,3,0,0
2,0,-0.358000,-0.214800,-3.046320,4,1,0
3,0,-0.358000,-0.214800,-3.046320,4,1,0
4,0,-0.358000,-0.214800,-3.046320,4,1,0
5,0,-0.358000,-0.214800,-3.046320,4,1,0
6,0,-0.358000,-0.214800,-3.046320,4,1,0
7,0,-0.358001,-0.214800,-3.046320,4,1,0
8,0,-0.358001,-0.214800,-3.046320,4,1,0
9,0,-0.358001,-0.214800,-3.046320,4,1,0


In [3]:
df.tail(10)

,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
20433104,0,0.163385,-0.114518,0.351625,71,1,0
20433105,0,0.163385,-0.114517,0.351644,71,1,0
20433106,0,0.163386,-0.114511,0.351659,71,1,0
20433107,0,0.163387,-0.114505,0.351676,71,1,0
20433108,0,0.163388,-0.114505,0.351684,71,1,0
20433109,0,0.163392,-0.114498,0.351691,71,1,0
20433110,0,0.163393,-0.114491,0.351706,71,1,0
20433111,0,0.163394,-0.114490,0.351717,71,1,0
20433112,0,0.163393,-0.114478,0.351747,71,1,0
20433113,0,0.163388,-0.114479,0.351771,71,1,0


In [4]:
#Sample the data
SAMPLING_RATE = 250 #Keep 1 row for every SAMPLING_RATE
df_sampled = df.iloc[::SAMPLING_RATE,:]
print("Size of new DF is {}".format(df_sampled.shape))
df_sampled.head(10)

Size of new DF is (81733, 7)


,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
0,0,0.000000,0.000000,0.00000,3,1,0
250,0,-0.356559,-0.214212,-3.07508,4,1,0
500,0,-0.376360,-0.238552,-3.11855,4,1,0
750,0,-0.336008,-0.245850,2.78610,4,1,0
1000,0,-0.315135,-0.257660,2.40072,4,1,0
1250,0,-0.335965,-0.256508,2.08609,4,1,0
1500,0,-0.312124,-0.264341,1.76586,4,1,0
1750,0,-0.306638,-0.283553,1.42170,4,1,0
2000,0,-0.346649,-0.284080,1.40252,4,1,0
2250,0,-0.342885,-0.297310,1.23708,4,1,0


In [5]:
#Break data into 3 parts. S = {Near 2 Walls, Near 1 Wall, Near No Walls}
#df2 = df_sampled.loc[df_sampled['ResetID'] % 3 == 2]
#df1 = df_sampled.loc[df_sampled['ResetID'] % 3 == 1]
#df0 = df_sampled.loc[df_sampled['ResetID'] % 3 == 0]
#df = df0

In [6]:
#Clean up the data
df_clean = df_sampled.copy()

df_clean.index = range(df_clean.shape[0])
#When the gazebo run it may collect some data of the robots when they aren't moving for the first few 50 or so samples.
#TODO
df_clean.head()

,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
0,0,0.000000,0.000000,0.00000,3,1,0
1,0,-0.356559,-0.214212,-3.07508,4,1,0
2,0,-0.376360,-0.238552,-3.11855,4,1,0
3,0,-0.336008,-0.245850,2.78610,4,1,0
4,0,-0.315135,-0.257660,2.40072,4,1,0


In [7]:
#Clean up the data

#Shift Yaw to go from 0 to 2pi, so just add PI since it currently goes for -pi to pi

df_clean['Yaw'] += np.pi
#The following is added so we can better discretize into 0, 180, 360, 480 degrees
#df_clean['Yaw'] -= (np.pi / 4.0)
#if(df_clean['Yaw'].max() > 2 * np.pi or df_clean['Yaw'].min() < 0):
#    print("[ERROR] Cleaning Yaw failed. Make sure range is from 0 and 2 * pi")
#    print("Yaw = ({} - {})".format(df_clean['Yaw'].min(), df_clean['Yaw'].max()))


df_clean.describe()

,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
count,81733.0,81733.000000,81733.000000,81733.000000,81733.000000,81733.0,81733.0
mean,0.0,-0.058978,-0.065501,2.917663,37.442196,1.0,0.0
std,0.0,0.212990,0.201395,1.793192,19.595887,0.0,0.0
min,0.0,-0.497524,-0.462529,0.000013,3.000000,1.0,0.0
25%,0.0,-0.234206,-0.213817,1.401143,20.000000,1.0,0.0
50%,0.0,-0.061629,-0.099530,2.896768,37.000000,1.0,0.0
75%,0.0,0.121773,0.101962,4.393763,54.000000,1.0,0.0
max,0.0,0.498124,0.399039,6.283123,71.000000,1.0,0.0


In [8]:
#Clean up the data

#Shift X and Y over so that way it could be made easier to use. Currently the world reference is at the center of the "play area"
LENGTH_OF_BOX = 1.127 #This can be obtained from the .sdf file of the weazelball enclosure in gazebo


df_clean['X'] += LENGTH_OF_BOX / 2
df_clean['Y'] += LENGTH_OF_BOX / 2
if (df_clean['X'].max() > LENGTH_OF_BOX or df_clean['Y'].max() > LENGTH_OF_BOX or df_clean['X'].min() < 0 or df_clean['Y'].min() < 0):
    print("[ERROR] Cleaning X/Y failed, Make sure the points are between 0 and LENGTH_OF_BOX")
    print("Y = ({} - {}) X = ({} to {})".format(df_clean['Y'].min(), df_clean['Y'].max(), df_clean['X'].min(), df_clean['X'].max()))
df_clean.head()


,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
0,0,0.563500,0.563500,3.141593,3,1,0
1,0,0.206941,0.349288,0.066513,4,1,0
2,0,0.187140,0.324948,0.023043,4,1,0
3,0,0.227492,0.317650,5.927693,4,1,0
4,0,0.248365,0.305840,5.542313,4,1,0


In [9]:
#Clean up the data

#Clean the time data since gazebo prints it weird...
#TODO

In [10]:
#Discretize the data
NUMBER_OF_SQUARES = 100 #This should be a square number to create equal sized squares.
RESOLUTION_OF_S1 = 100 #This is used to discretize the yaw angle over 0 - 2*pi

df_discretized = df_clean.copy()
    
mappingBoxConstant = (NUMBER_OF_SQUARES ** (1/2.0)) / (LENGTH_OF_BOX)
for index, row in df_clean.iterrows():
    df_discretized.at[index, 'X'] = int(row['X'] * mappingBoxConstant)
    df_discretized.at[index, 'Y'] = int(row['Y'] * mappingBoxConstant)
    df_discretized.at[index, 'Yaw'] = int(row['Yaw'] / RESOLUTION_OF_S1) * RESOLUTION_OF_S1
        
df_discretized.describe()

,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
count,81733.0,81733.000000,81733.000000,81733.0,81733.000000,81733.0,81733.0
mean,0.0,3.958499,3.940367,0.0,37.442196,1.0,0.0
std,0.0,1.911782,1.810504,0.0,19.595887,0.0,0.0
min,0.0,0.000000,0.000000,0.0,3.000000,1.0,0.0
25%,0.0,2.000000,3.000000,0.0,20.000000,1.0,0.0
50%,0.0,4.000000,4.000000,0.0,37.000000,1.0,0.0
75%,0.0,6.000000,5.000000,0.0,54.000000,1.0,0.0
max,0.0,9.000000,8.000000,0.0,71.000000,1.0,0.0


In [11]:
#Verify Discretizing suceeded by checking that number of states generated is the number of states we expeted or less (Sometimes these things dont visit all states)

if (df_discretized['X'].max() > (NUMBER_OF_SQUARES ** (1/2.0)) or df_discretized['Y'].max() > (NUMBER_OF_SQUARES ** (1/2.0)) or df_discretized['X'].min() < 0 or df_discretized['Y'].min() < 0):
    print("[ERROR] Discretizing X/Y failed, Make sure the points are between 0 and (NUMBER_OF_SQUARES ** (1/2.0)")
    print("Y = ({} - {}) X = ({} to {})".format(df_clean['Y'].min(), df_clean['Y'].max(), df_clean['X'].min(), df_clean['X'].max()))


In [12]:
TRANSLATION_MATRIX_INITIAL_VALUE = 1
#Create the matrix representing the Markov Chain
#I am assuming we are discretizing the space into equal sized boxes
#The transition matrix A is of size 
#( # of states of X * # of states of Y * # of states of Yaw)
#( # of states of X and Y = mappingBoxConstant * LENGTH_OF_BOX )
#( # of states of Yaw = int(2*pi / RESOLUTION_OF_S1)+1)
number_of_x_states = mappingBoxConstant * LENGTH_OF_BOX
number_of_y_states = mappingBoxConstant * LENGTH_OF_BOX
number_of_s1_states = int(2*np.pi / RESOLUTION_OF_S1) + 1

n = int(number_of_x_states * number_of_y_states * number_of_s1_states)
print("[DEBUG] Size of n is {}".format(n))
translation_matrix = pd.DataFrame(0, index=range(n), columns=range(n))#We use 1 here to set the whole matrix elements to 1
translation_matrix.head()

[DEBUG] Size of n is 100


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
print(number_of_s1_states)

1


In [14]:


#The formula for mapping a 3D array to 1D is
#(z * xMax * yMax) + (y * xMax) + x;
#https://stackoverflow.com/questions/7367770/how-to-flatten-or-index-3d-array-in-1d-array
X_MAX = NUMBER_OF_SQUARES ** (1/2.0)
Y_MAX = NUMBER_OF_SQUARES ** (1/2.0)
YAW_MAX = (2 * np.pi) / RESOLUTION_OF_S1
YAW_MAX += 0
mapping = Mapping(X_MAX, Y_MAX, YAW_MAX)#Fill in the logical areas that the system can reach (For now I am assuming it can go up/down 2 yaw states or the surronding (x,y) blocks)
print("XMAX = {}, YMAX = {} YAWMAX = {}".format(X_MAX,Y_MAX,YAW_MAX))

XMAX = 10.0, YMAX = 10.0 YAWMAX = 0.0628318530718


In [15]:
#HUERISTIC: Add a +1 to any logical possible state the structure would likely end up in.
#This lowers the amount of artifiical data in the matrix (most of which isnt needed)



for index, row in translation_matrix.iterrows():
    if(TRANSLATION_MATRIX_INITIAL_VALUE == 0):
        break
    (x,y,yaw) = mapping.map1Dto3D(index)
    #Generate all possible (x,y,yaw) permutations
    #I assume we can only move with 1 around (x,y) including diagnols and +- 2 yaw states
    VARIATION_OF_X = 1
    VARIATION_OF_Y = 1
    VARIATION_OF_YAW = 1
    possible_spots = []
    for x_p in range(-VARIATION_OF_X,VARIATION_OF_X+1):
        for y_p in range(-VARIATION_OF_Y,VARIATION_OF_Y+1):
            for yaw_p in range(-VARIATION_OF_YAW,VARIATION_OF_YAW+1):
                if(mapping.checkValid3DMap(x+x_p, y+y_p, yaw+yaw_p)):
                    possible_spots.append( (x+x_p, y+y_p, yaw+yaw_p) )
    for pose in possible_spots:
        translation_matrix.at[mapping.map3Dto1D(x,y,yaw), mapping.map3Dto1D(pose[0],pose[1],pose[2])] = translation_matrix.at[mapping.map3Dto1D(x,y,yaw), mapping.map3Dto1D(pose[0],pose[1],pose[2])] +TRANSLATION_MATRIX_INITIAL_VALUE
translation_matrix.head()    

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
#Create a dictionary for storing the transition states analysis
d ={}
#Our keys to the dictionary will look like (x_t, y_t, yaw_t, x_t+1, y_t+1, yaw_t+1)
#Go through all but last row since t+1 isnt defined there...
skipCount = 0
try:
    for index, row in df_discretized.iterrows():
        if(index == df_discretized.index[-1]):
            break
        if(df_discretized.at[index, 'ResetID'] != df_discretized.at[index + 1, 'ResetID']):
            skipCount += 1
            continue
        #Need to round here because Yaw data has floating point error
        key = (df_discretized.at[index, 'X'], df_discretized.at[index, 'Y'], round(df_discretized.at[index, 'Yaw'], 6),df_discretized.at[index+1, 'X'], df_discretized.at[index+1, 'Y'], round(df_discretized.at[index+1, 'Yaw'], 6) )

        if key in d:
            d[key] += 1
        else:
            d[key] = 1
except Exception as e:
    print e
    
print "[DEBUG] Skipped {} events".format(skipCount)
d

[DEBUG] Skipped 68 events


{(0.0, 2.0, 0.0, 1.0, 2.0, 0.0): 1,
 (0.0, 3.0, 0.0, 0.0, 2.0, 0.0): 1,
 (0.0, 3.0, 0.0, 0.0, 3.0, 0.0): 978,
 (0.0, 3.0, 0.0, 1.0, 3.0, 0.0): 2,
 (0.0, 4.0, 0.0, 0.0, 4.0, 0.0): 13,
 (0.0, 4.0, 0.0, 0.0, 5.0, 0.0): 1,
 (0.0, 4.0, 0.0, 1.0, 4.0, 0.0): 3,
 (0.0, 5.0, 0.0, 0.0, 5.0, 0.0): 2,
 (0.0, 5.0, 0.0, 1.0, 5.0, 0.0): 1,
 (1.0, 1.0, 0.0, 1.0, 1.0, 0.0): 20,
 (1.0, 1.0, 0.0, 1.0, 2.0, 0.0): 2,
 (1.0, 1.0, 0.0, 2.0, 1.0, 0.0): 4,
 (1.0, 2.0, 0.0, 0.0, 3.0, 0.0): 1,
 (1.0, 2.0, 0.0, 1.0, 2.0, 0.0): 1067,
 (1.0, 2.0, 0.0, 1.0, 3.0, 0.0): 6,
 (1.0, 2.0, 0.0, 2.0, 1.0, 0.0): 1,
 (1.0, 2.0, 0.0, 2.0, 2.0, 0.0): 13,
 (1.0, 3.0, 0.0, 0.0, 3.0, 0.0): 3,
 (1.0, 3.0, 0.0, 1.0, 2.0, 0.0): 5,
 (1.0, 3.0, 0.0, 1.0, 3.0, 0.0): 1421,
 (1.0, 3.0, 0.0, 1.0, 4.0, 0.0): 11,
 (1.0, 3.0, 0.0, 2.0, 3.0, 0.0): 3,
 (1.0, 4.0, 0.0, 0.0, 4.0, 0.0): 3,
 (1.0, 4.0, 0.0, 1.0, 3.0, 0.0): 11,
 (1.0, 4.0, 0.0, 1.0, 4.0, 0.0): 1299,
 (1.0, 4.0, 0.0, 1.0, 5.0, 0.0): 6,
 (1.0, 4.0, 0.0, 2.0, 4.0, 0.0): 4,
 (1.0, 5.0, 

In [17]:

#Fill in matrix with dictionary data


for key, value in d.iteritems():
    #we need to map yaw to an int state
    element_t = mapping.map3Dto1D(key[0], key[1], int(key[2]/RESOLUTION_OF_S1))
    element_t_plus_1 = mapping.map3Dto1D(key[3], key[4], int(key[5]/RESOLUTION_OF_S1))
    #Use the following to verify we the math above is fine
    if((mapping.checkValid1DMap(element_t)) & (mapping.checkValid1DMap(element_t_plus_1)) == 0 ):
        print "[ERROR] BAD MAPPING!"
    try:
        translation_matrix.at[element_t, element_t_plus_1] = value + translation_matrix.at[element_t, element_t_plus_1]
    except Exception as e:
        print("error = {}".format(e))
        print("element _t = {}".format(element_t))
        print("element _t = {}".format(mapping.map1Dto3D(element_t)))
        print("element_t+1 = {}".format(element_t_plus_1))
        print("element _t+1 = {}".format(mapping.map1Dto3D(element_t_plus_1)))
    # print("key = {}, elements = {}, {}".format(key, element_t, element_t_plus_1))


[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD M

In [18]:
foo = d.keys()
from operator import itemgetter
min(foo,key=itemgetter(2))

(5.0, 4.0, 0.0, 6.0, 4.0, 0.0)

In [19]:
#Check sum of "events" per matrix

totalEvents = 0
for index,row in translation_matrix.iterrows():
    totalEvents += row.sum()
print("Total Events is {}".format(totalEvents))
print("Size of data point df is {}".format(df_discretized.size))

Total Events is 81698
Size of data point df is 572131


In [20]:
#Divide the whole dataframe by number of data collections to get the probabilities.


magnitudeVector = pd.Series(0, index=range(n + 1))



for index, row in translation_matrix.iterrows():
    totalActionsInThisState = row.sum()
    magnitudeVector.iloc[index] = totalActionsInThisState
    if totalActionsInThisState == 0:
        continue
    translation_matrix.iloc[index] /= totalActionsInThisState

translation_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.333333,0.333333,0.333333,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.333333,0.333333,0.333333,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
#validate the matrix (all rows == 1)
for index, row in translation_matrix.iterrows():
    if(abs(row.sum() - 1.0) > FLOAT_ERROR_TOLERANCE):
        print("[ERROR] Row probability not equal to one!")
        print(index)
        print(row.sum())


[ERROR] Row probability not equal to one!
7
0.0
[ERROR] Row probability not equal to one!
8
0.0
[ERROR] Row probability not equal to one!
9
0.0
[ERROR] Row probability not equal to one!
19
0.0
[ERROR] Row probability not equal to one!
58
0.0
[ERROR] Row probability not equal to one!
59
0.0
[ERROR] Row probability not equal to one!
60
0.0
[ERROR] Row probability not equal to one!
68
0.0
[ERROR] Row probability not equal to one!
69
0.0
[ERROR] Row probability not equal to one!
70
0.0
[ERROR] Row probability not equal to one!
78
0.0
[ERROR] Row probability not equal to one!
79
0.0
[ERROR] Row probability not equal to one!
80
0.0
[ERROR] Row probability not equal to one!
81
0.0
[ERROR] Row probability not equal to one!
82
0.0
[ERROR] Row probability not equal to one!
83
0.0
[ERROR] Row probability not equal to one!
85
0.0
[ERROR] Row probability not equal to one!
86
0.0
[ERROR] Row probability not equal to one!
87
0.0
[ERROR] Row probability not equal to one!
88
0.0
[ERROR] Row probability

In [22]:
#Make matrix into CSV
translation_matrix.to_csv(data_prefix + 'markov4_100.csv', encoding='utf-8', index=False)


In [23]:
#Make csv of the number of instances for each row
#magnitudeVector.to_csv(data_prefix + 'magnitude_vector_out.csv', encoding='utf-8', index=False)


In [24]:
print("Total time taken = {}".format(time.time()-start_time))

Total time taken = 79.2445540428
